In [1]:
import os
import pandas as pd
import numpy as np
import math
import datetime as dt
import matplotlib.pyplot as plt

In [2]:
# Evaluation
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

In [6]:
!pip install tensorflow

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM

In [8]:
#plotting
import matplotlib.pyplot as plt
from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [9]:
df=pd.read_csv("BTC-USD.csv")

In [10]:
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
1,2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2,2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
3,2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
4,2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100
...,...,...,...,...,...,...,...
3574,2024-06-30,60888.445313,62892.828125,60632.949219,62678.292969,62678.292969,17333226409
3575,2024-07-01,62673.605469,63777.226563,62495.511719,62851.980469,62851.980469,25468379421
3576,2024-07-02,62844.410156,63203.359375,61752.746094,62029.015625,62029.015625,20151616992
3577,2024-07-03,62034.332031,62187.703125,59419.386719,60173.921875,60173.921875,29756701685


In [11]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
1,2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2,2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
3,2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
4,2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100


In [12]:
print("Total number of days present in dataset:",df.shape[0])
print("Total number of fields present in the dataset:",df.shape[1])

Total number of days present in dataset: 3579
Total number of fields present in the dataset: 7


In [13]:
df.shape

(3579, 7)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3579 entries, 0 to 3578
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       3579 non-null   object 
 1   Open       3579 non-null   float64
 2   High       3579 non-null   float64
 3   Low        3579 non-null   float64
 4   Close      3579 non-null   float64
 5   Adj Close  3579 non-null   float64
 6   Volume     3579 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 195.9+ KB


In [15]:
df.describe()

,Open,High,Low,Close,Adj Close,Volume
count,3579.000000,3579.000000,3579.000000,3579.000000,3579.000000,3.579000e+03
mean,16910.465457,17296.261910,16497.574866,16925.343047,16925.343047,1.731818e+10
std,18802.581314,19227.012873,18334.033821,18810.986298,18810.986298,1.917512e+10
min,176.897003,211.731003,171.509995,178.102997,178.102997,5.914570e+06
25%,1052.405029,1079.729981,1037.085022,1054.325012,1054.325012,2.510190e+08
50%,8852.780273,9081.279297,8677.316406,8864.766602,8864.766602,1.328114e+10
75%,27796.959961,28336.328125,27253.883789,27808.447265,27808.447265,2.790441e+10
max,73079.375000,73750.070313,71334.093750,73083.500000,73083.500000,3.509679e+11


In [17]:
df.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [19]:
df.shape

(3579, 7)

In [21]:
sd=df.iloc[0][0]
ed=df.iloc[-1][0]
print("Starting date:",sd)
print("End date:",ed)

Starting date: 2014-09-17
End date: 2024-07-04


C:\Users\89339\AppData\Local\Temp\ipykernel_29136\1282837128.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sd=df.iloc[0][0]
C:\Users\89339\AppData\Local\Temp\ipykernel_29136\1282837128.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ed=df.iloc[-1][0]
